In [1]:
import numpy as np
import pandas as pd

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [15]:
def merge2cats(cat1, cat2, merge_dist=400/3600, name='', prms={}): 
    from astropy.coordinates import SkyCoord
    from astropy import units as u
    
    sc1 = SkyCoord(ra=np.array(cat1['RA']) * u.degree, dec=np.array(cat1['DEC']) * u.degree, frame='icrs')
    sc2 = SkyCoord(ra=np.array(cat2['RA']) * u.degree, dec=np.array(cat2['DEC']) * u.degree, frame='icrs')
    
    idx, d2d, _ = sc1.match_to_catalog_sky(sc2)
    matched = d2d.degree <= merge_dist
    print(len(cat1), 'obj in 1')
    print(len(cat2), 'obj in 2')
    print(np.count_nonzero(matched), 'matched')
    print(len(cat1) - np.count_nonzero(matched), 'unique from 1')
    
    for coord in ['RA', 'DEC']:
        cat1[name+coord] = np.nan
        cat2[name+coord] = np.nan
        cat1[name+coord].iloc[matched] = np.array(cat2[coord][idx[matched]])
    
    for prm in prms:
        cat1[prm] = prms[prm]
        cat1[prm].iloc[matched] = np.array(cat2[prm][idx[matched]])
    
    print(len(cat2) - np.count_nonzero(matched), 'unique from 2')
    cat2 = cat2.loc[idx[d2d.degree > merge_dist]]
    print(len(cat2), 'unique from 2')
    
    return pd.concat([cat1, cat2])

In [16]:
c = merge2cats(pd.read_csv('/home/rt2122/Data/clusters/planck_z.csv'), pd.read_csv('/home/rt2122/Data/original_catalogs/csv/MCXC.csv'))

1094 obj in 1
1743 obj in 2
545 matched
549 unique from 1
1198 unique from 2
549 unique from 2


/home/rt2122/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [17]:
c

,Name,RA,DEC,z,M500,MCXC,RedMAPPer
0,PSZ2 G000.04+45.13,NaN,NaN,0.1198,3.962411,J1516.5-0056,RMJ151653.9-010506.3
1,PSZ2 G000.13+78.04,NaN,NaN,0.1710,5.122391,J1334.1+2013,RMJ133408.7+201453.0
2,PSZ2 G000.40-41.86,NaN,NaN,0.1651,5.297053,J2104.3-4120,
3,PSZ2 G000.77-35.69,NaN,NaN,0.3416,6.333562,J2031.8-4037,
4,PSZ2 G002.08-68.28,NaN,NaN,0.1400,2.835297,,
...,...,...,...,...,...,...,...
1617,NaN,NaN,NaN,0.1130,1.252100,NaN,NaN
1451,NaN,NaN,NaN,0.0774,1.858500,NaN,NaN
1122,NaN,NaN,NaN,0.0263,0.735200,NaN,NaN
1060,NaN,NaN,NaN,0.1251,2.206400,NaN,NaN
